In [4]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer
encoder = SentenceTransformer("intfloat/multilingual-e5-small")

/home/rishabh/scrapy/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/rishabh/scrapy/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
# load json file as list of dictionaries
import json
with open('diga_crawler/output.json') as f:
    data = json.load(f)

In [6]:
data[32]

{'name': 'Selfapys Online-Kurs bei Binge-Eating-Störung',
 'description': 'Der Selfapy Online-Kurs ist eine digitale Anwendung für Betroffene einer Binge-Eating-Störung. Der Kurs vermittelt Methoden und Techniken basierend auf der kognitiven Verhaltenstherapie und unterstützt bei der Durchführung sowie der Dokumentation der Übungen, mit dem Ziel, die Symptomatik des Anwendenden zu verbessern.',
 'anzuwenden_bei': 'F50.4 Essattacken bei anderen psychischen Störungen F50.8 Sonstige Essstörungen F50.9 Essstörung, nicht näher bezeichnet Mehr erfahren'}

In [7]:
client = QdrantClient(":memory:")

In [8]:
client.recreate_collection(
    collection_name="digas",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(),  # Vector size is defined by used model
        distance=models.Distance.COSINE,
    ),
)

/tmp/ipykernel_1632/3256227066.py:1: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


True

In [9]:
client.upload_points(
    collection_name="digas",
    points=[
        models.PointStruct(
            id=idx, vector=encoder.encode(doc["anzuwenden_bei"]).tolist(), payload=doc
        )
        for idx, doc in enumerate(data)
    ],
)

In [22]:
# Endometriose  
hits = client.search(
    collection_name="digas",
    query_vector=encoder.encode("Endometioe").tolist(),
    limit=1,
)
for hit in hits:
    payload = hit.payload
    score = hit.score
    print("Payload:")
    for key, value in payload.items():
        print(f"  {key}: {value}")
    print(f"Score: {score}")
    print("-" * 40)

Payload:
  name: Endo-App
  description: Die Endo-App ist ein digitales Medizinprodukt zur multimodalen Unterstützung von Endometriose-Betroffenen. Ziel des Programms ist es, die Nutzerinnen dabei zu unterstützen, im Rahmen des Selbstmanagements ihre Lebensqualität zu verbessern. Das Selbstmanagementkonzept beruht grundlegend auf einem Symptomtagebuch, in dem Nutzerinnen ihre Symptome, Aktivitäten, Übungen und ihre Behandlungen dokumentieren können, sowie auf multimodalen Inhalten. Als Teil der multimodalen Therapie werden wissensvermittelnde Elemente, validierte Fragebögen, geführte Übungen und Techniken aus dem Bereich Gynäkologie, Physiotherapie, Psychologie, Schmerztherapie, Rehabilitation, Sozialpädagogik, Ernährung, Ressourcenaktivierung sowie Resilienzstärkung zur Verfügung gestellt.
  anzuwenden_bei: N80 Endometriose Mehr erfahren
Score: 0.8614803201243191
----------------------------------------


In [31]:
# Semantic Search Example
# Zucker
# Diabetes
# Zuckerkrankheit


hits = client.search(
    collection_name="digas",
    query_vector=encoder.encode("").tolist(),
    limit=3,
)
for hit in hits:
    payload = hit.payload
    score = hit.score
    print("Payload:")
    for key, value in payload.items():
        print(f"  {key}: {value}")
    print(f"Score: {score}")
    print("-" * 40)

Payload:
  name: Kranus Lutera
  description: Kranus Lutera ist eine digitale Therapie für Männer mit Blasenentleerungsstörungen, auch “Lower Urinary Tract Symptoms” (kurz “LUTS”) genannt. Kranus Lutera bietet eine einzigartige, einfache und personalisierte Therapie über 12 Wochen, die darauf abzielt, Männern mit häufigem Harndrang bei BPH und OAB durch eine Vielzahl von Funktionen und personalisierten Ansätzen zu unterstützen.
  anzuwenden_bei: N32.8 Sonstige näher bezeichnete Krankheiten der Harnblase N40 Prostatahyperplasie Mehr erfahren
Score: 0.8684824625536445
----------------------------------------
Payload:
  name: Mawendo
  description: Mawendo stellt Trainingsprogramme mit Übungsvideos, Gesundheitsinformationen und Dokumentationsmöglichkeiten bereit. Das Trainingsprogramm wird initial durch Ärztinnen und Ärzte ausgewählt und in Bezug auf das Krankheitsbild, die spezifischen Übungen sowie die Trainingsphasen für die Patientin oder den Patienten individualisiert. Im Anschluss k